In [ ]:
import requests
from bs4 import BeautifulSoup
from lxml import etree
import pandas as pd
import queue
from threading import Thread as th

In [ ]:
r = requests.get("https://autos.mercadolibre.com.ar/autos_ITEM*CONDITION_2230581_NoIndex_True#unapplied_filter_id%3DCOLOR%26unapplied_filter_name%3DColor%26unapplied_value_id%3D52049%26unapplied_value_name%3DNegro%26unapplied_autoselect%3Dfalse")

In [ ]:
r.status_code


In [ ]:
soup = BeautifulSoup(r.content, 'html.parser')

In [ ]:
title = soup.find_all('h2',attrs={"class":"ui-search-item__title ui-search-item__group__element"})
title = [i.text for i in title]

In [ ]:
urls = soup.find_all('a', attrs={"class":"ui-search-result__content ui-search-link"})
urls = [i.get('href') for i in urls]

In [ ]:
# price = soup.find_all('span', attrs={"class":"price-tag-fraction"})#First 2 results are not valid! (At least in first page)
# price = [i.text for i in price][2:]

In [ ]:
dom = etree.HTML(str(soup))
price = dom.xpath('//div[@class="ui-search-result__content-wrapper"]//span[@class="price-tag-fraction"]')
price = [i.text for i in price]

In [ ]:
next_page = dom.xpath('//a[@class="andes-pagination__link ui-search-link"]')[0].get('href')

In [ ]:
next_page

In [ ]:
init = int(soup.find('span', attrs={"class":"andes-pagination__link"}).text)#gets first page
last = int(soup.find('li', attrs={"class":"andes-pagination__page-count"}).text.split(" ")[1])#gets last page count, needs some formatting "returns de 42"

In [ ]:
urls_list=[]
title_list=[]
price_list=[]

In [ ]:
next_page = "https://autos.mercadolibre.com.ar/autos_ITEM*CONDITION_2230581_NoIndex_True#unapplied_filter_id%3DCOLOR%26unapplied_filter_name%3DColor%26unapplied_value_id%3D52049%26unapplied_value_name%3DNegro%26unapplied_autoselect%3Dfalse"#init
while True:
    r = requests.get(next_page)
    if r.status_code == 200:
        soup = BeautifulSoup(r.content, 'html.parser')
        
        #title
        title = soup.find_all('h2',attrs={"class":"ui-search-item__title ui-search-item__group__element"})
        title = [i.text for i in title]
        title_list.extend(title)
        
        #url
        urls = soup.find_all('a', attrs={"class":"ui-search-result__content ui-search-link"})
        urls = [i.get('href') for i in urls]
        urls_list.extend(urls)
       
        #price
        dom = etree.HTML(str(soup))
        price = dom.xpath('//div[@class="ui-search-result__content-wrapper"]//span[@class="price-tag-fraction"]')
        price = [i.text for i in price]
        price_list.extend(price)
        
        #new pages
        init = int(soup.find('span', attrs={"class":"andes-pagination__link"}).text)#gets first page
        last = int(soup.find('li', attrs={"class":"andes-pagination__page-count"}).text.split(" ")[1])#gets last page count, needs some formatting "returns de 42"
    else:
        break
    print(init, last)
    if init == last:
        break
    next_page = dom.xpath('//li[@class="andes-pagination__button andes-pagination__button--next"]//a[@class="andes-pagination__link ui-search-link"]')[0].get('href')#specify NEXT btn if not goes back and forth

In [ ]:
#CHecks that it got the same amount of data for each item
print([len(urls_list),
len(title_list),
len(price_list)])

In [ ]:
df = pd.DataFrame({"Car":title_list, "url":urls_list, "price":price_list})

In [ ]:
df

#### This DataFrame only contains the data from the general used cars, if we want more data, we need to dive deeper, for example we can loop for each car brand in the url

In [ ]:
brands = ['Alfa-Romeo',
'Audi',
'BMW',
'Chery',
'Chevrolet',
'Chrysler',
'Citroen',
'Dodge',
'DS',
'Fiat',
'Ford',
'Honda',
'Hyundai',
'Isuzu',
'Jeep',
'Kia',
'Land-Rover',
'Mazda',
'Mercedes-Benz',
'Mini',
'Mitsubishi',
'Nissan',
'Peugeot',
'Porsche',
'RAM',
'Renault',
'Smart',
'Subaru',
'Suzuki',
'Toyota',
'Volkswagen',
'Volvo']
print(f"We have {len(brands)} different brands")
brand_url = f"https://autos.mercadolibre.com.ar/{brands[5]}/autos_ITEM*CONDITION_2230581_NoIndex_True#applied_filter_id%3DBRAND%26applied_filter_name%3DMarca%26applied_filter_order%3D2%26applied_value_id%3D60249%26applied_value_name%3DVolkswagen%26applied_value_order%3D36%26applied_value_results%3D18220%26is_custom%3Dfalse"

### now we try to retireve more info from each of the 2592 cars

In [ ]:
#Given car
given_url = "https://auto.mercadolibre.com.ar/MLA-1125791437-renaul-duster-dynamique-16-ano-2016-as-automobili-_JM#position=1&search_layout=grid&type=item&tracking_id=bc9cdbb4-84a2-4ce2-a3ec-e18a58b86156"
r = requests.get(given_url)
r.status_code

In [ ]:
soup = BeautifulSoup(r.content, 'html.parser')

In [ ]:
row = soup.find_all('span', attrs={"class":"andes-table__column--value"})#gets all the info we want with this class!!
row = [i.text for i in row]
col_titles = soup.find_all('th', attrs={"class":"andes-table__header andes-table__header--left ui-pdp-specs__table__column ui-pdp-specs__table__column-title"})#gets titles
col_titles = [i.text for i in col_titles]
#extra data (Location)
location = soup.find_all('p', attrs={"class":"ui-seller-info__status-info__subtitle"})[2].text
price = int(soup.find_all('span', attrs={"class":"andes-money-amount__fraction"})[0].text.replace(".",""))#We also do some data cleaning here..

In [ ]:
df = pd.DataFrame(columns=col_titles)
df_length = len(df)
df.loc[df_length] = row
df["Location"] = location
df["Price"] = price
df["Link"] = given_url #In case we want to check it out, maybe we can find deals too
df

So far so good, we got the data we want for 1 car, now we need it for all the cars we can get
### The website lists more than 100000 cars, but it will only present 2592 per filter, so you need to scrap every filter...

##### We are going to combine the methods and scrap over all brands

In [ ]:
urls_list=[]
title_list=[]
price_list=[]
for i in range(len(brand_url)):
    brand_url = f"https://autos.mercadolibre.com.ar/{brands[i]}/autos_ITEM*CONDITION_2230581_NoIndex_True#applied_filter_id%3DBRAND%26applied_filter_name%3DMarca%26applied_filter_order%3D2%26applied_value_id%3D60249%26applied_value_name%3DVolkswagen%26applied_value_order%3D36%26applied_value_results%3D18220%26is_custom%3Dfalse"
    next_page = brand_url#init
    print(brands[i])
    while True:
        r = requests.get(next_page)
        if r.status_code == 200:
            soup = BeautifulSoup(r.content, 'html.parser')
            print(r.status_code)
               #title
            title = soup.find_all('h2',attrs={"class":"ui-search-item__title ui-search-item__group__element"})
            title = [i.text for i in title]
            title_list.extend(title)

            #url
            urls = soup.find_all('a', attrs={"class":"ui-search-result__content ui-search-link"})
            urls = [i.get('href') for i in urls]
            urls_list.extend(urls)

            #price
            dom = etree.HTML(str(soup))
            price = dom.xpath('//div[@class="ui-search-result__content-wrapper"]//span[@class="price-tag-fraction"]')
            price = [i.text for i in price]
            price_list.extend(price)
            #new pages
            init = int(soup.find('span', attrs={"class":"andes-pagination__link"}).text)#gets first page
            last = int(soup.find('li', attrs={"class":"andes-pagination__page-count"}).text.split(" ")[1])#gets last page count, needs some formatting "returns de 42"
        else:
            break
        print(init, last)
        if init == last:
            break
        next_page = dom.xpath('//li[@class="andes-pagination__button andes-pagination__button--next"]//a[@class="andes-pagination__link ui-search-link"]')[0].get('href')#specify NEXT btn if not goes back and forth

In [ ]:
r = requests.get("https://autos.mercadolibre.com.ar/autos_ITEM*CONDITION_2230581_NoIndex_True#filter")
soup = BeautifulSoup(r.content, 'html.parser')
soup.find_all('a', attrs={"class":"ui-search-search-modal-filter ui-search-link"})

###### Now we get (after some time of scraping) Links to 33951 cars

In [ ]:
df = pd.DataFrame({"Car":title_list, "url":urls_list, "price":price_list})

In [ ]:
df.to_csv("car_urls")
df

In [ ]:
df = pd.read_csv("car_urls")# to load in case there is a problem
df

We will iterate over this links to get the full data of each car
### Note: We had to make some changes, since not always all data was available (e.g color) so it needed to match number of items in col

In [ ]:
#Given car
#list_of_dicts = [] #I run this the first time then comment, since some times it crashes and need to start from were it left
# for given_url in df["url"][4705:]:
def car(given_url):
    r = requests.get(given_url)
    r.status_code 
    print(f"Code:{r.status_code} ok scrapping {given_url}")
    soup = BeautifulSoup(r.content, 'html.parser')
    row = soup.find_all('span', attrs={"class":"andes-table__column--value"})#gets all the info we want with this class!!
    row = [i.text for i in row]
    if row == []:
        return
    
    col_titles = soup.find_all('th', attrs={"class":"andes-table__header andes-table__header--left ui-pdp-specs__table__column ui-pdp-specs__table__column-title"})#gets titles
    col_titles = [i.text for i in col_titles]
    col_titles.append('Location')
    col_titles.append('Price')
    col_titles.append('Link')
    
    #extra data (Location)
    location = soup.find_all('p', attrs={"class":"ui-seller-info__status-info__subtitle"})
    if location == []:
        print("no loc, passed")
        return
        
    location = location[-1].text
    price = int(soup.find_all('span', attrs={"class":"andes-money-amount__fraction"})[0].text.replace(".",""))#We also do some data cleaning here..
    
    row.append(location)
    row.append(price)
    row.append(given_url)
    
    kv_pairs = dict(zip(col_titles, row))
    return kv_pairs

# threads = [] 
# list2 = []
# que = queue.Queue()
# a = 4705
# while a < 33951:
#     threads = []
#     for i in range(0,10):
#         given_url = df["url"][i+a]
#         t = th(target=lambda q, arg1: q.put(car(arg1)), args=(que, given_url))
#         t.start()
#         threads.append(t)
#         a = a + 1
#     for t in threads:
#         t.join()

#         list2.append(que.get())
#     print(f" URL: {a} / 33951")
a = 8572
for given_url in df["url"][a::3]:# WE increment step by 3 because it scraps 1000 urls per hour, so we can start working, later we will scrap all
    kv_pairs = car(given_url)
    if kv_pairs == None:
        continue
    list_of_dicts.append(kv_pairs)
    a = a + 3
    print(f" URL: {a} / 33951")

### So threading wasnt optimizing speed, scrapping is time consuming, but is a one time operation so is ok.

In [ ]:
list_of_dicts_bk=list_of_dicts

In [ ]:
pd.DataFrame(list_of_dicts)#.to_csv("Autos_citroen_final.csv")

In [ ]:
df

Finnally we concat all df that we got into the final df and save it as csv, the result contains 15741 cars out of 33950 in the list, we can keep scraping the rest meanwhile, but this is ok as a starting point.

In [ ]:
da4

### Next steps:
* Clean DF
* Upload DF to AWS DB

In [ ]:
#Remaining URLS to scrap
for link in da4["Link"]:
      print(link in df['url'].values)

In [ ]:
urls = pd.read_csv("tosqldb.csv")
urls = urls["url"]
urls = urls.apply(lambda x : x[:50])

In [ ]:
#Given car
list_of_dicts = []
# for given_url in df["url"][4705:]:
def car(given_url):
    r = requests.get(given_url)
    r.status_code 
    print(f"Code:{r.status_code} ok scrapping {given_url}")
    soup = BeautifulSoup(r.content, 'html.parser')
    row = soup.find_all('span', attrs={"class":"andes-table__column--value"})#gets all the info we want with this class!!
    row = [i.text for i in row]
    if row == []:
        return
    
    col_titles = soup.find_all('th', attrs={"class":"andes-table__header andes-table__header--left ui-pdp-specs__table__column ui-pdp-specs__table__column-title"})#gets titles
    col_titles = [i.text for i in col_titles]
    col_titles.append('Location')
    col_titles.append('Price')
    col_titles.append('Link')
    
    #extra data (Location)
    location = soup.find_all('p', attrs={"class":"ui-seller-info__status-info__subtitle"})
    if location == []:
        print("no loc, passed")
        return
        
    location = location[-1].text
    price = int(soup.find_all('span', attrs={"class":"andes-money-amount__fraction"})[0].text.replace(".",""))#We also do some data cleaning here..
    
    row.append(location)
    row.append(price)
    row.append(given_url)
    
    kv_pairs = dict(zip(col_titles, row))
    return kv_pairs


a = 0
b = len(df["url"].values) 
for given_url in df["url"]:
    if given_url[:50] in urls.values:
        print(f" URL: {given_url} Already scraped, continue")
        a = a + 1
        continue
    kv_pairs = car(given_url)
    if kv_pairs == None:
        continue
    list_of_dicts.append(kv_pairs)
    a = a + 1
    print(f" URL: {a} / {b}")

In [ ]:
df0 = pd.DataFrame(list_of_dicts)

In [ ]:
df0.to_csv('last_scrap.csv')

In [ ]:
urls = pd.read_csv("tosqldb.csv")
urls = urls["url"]
urls

In [ ]:
urls.isin(df["url"])
df["url"][0][:50] in urls.values

In [ ]:
urls = urls.apply(lambda x : x[:50])

In [ ]:
given_url in urls.values